# Deep Learning Workshop - Assignment 1, Question 3

## Installations and imports

In [1]:
!pip install lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 33.8 MB/s eta 0:00:00a 0:00:01


In [2]:
import os
import time
import torch
import torchvision
import lightning as L
from torch import optim, nn
from torch.utils.data import DataLoader
from torchvision import models, transforms
from torchvision.datasets import ImageFolder
from tqdm import tqdm
from lightning.pytorch.callbacks import LearningRateMonitor, ModelCheckpoint, EarlyStopping
from lightning.pytorch.loggers import TensorBoardLogger
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

## Constants

In [3]:
BATCH_SIZE = 64
TRAIN = 'train'
TEST = 'test'
VALID = 'valid'
BASE_FOLDER = '/kaggle/input/100-bird-species'

## Define Transforms

In [4]:
# Define transforms for train and test sets, according to ImageNet statistics
train_means = [0.485, 0.456, 0.406]
train_stds = [0.229, 0.224, 0.225]
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=train_means, std=train_stds)
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=train_means, std=train_stds)
])

## Load data into Datasets using ImageFolder

In [5]:
train_dataset = ImageFolder(root=f'{BASE_FOLDER}/{TRAIN}', transform=train_transform)
test_dataset = ImageFolder(root=f'{BASE_FOLDER}/{TEST}', transform=test_transform)
valid_dataset = ImageFolder(root=f'{BASE_FOLDER}/{VALID}', transform=test_transform)

## Create DataLoaders

In [6]:
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=3)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=3)
val_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=3)

# Sections A, B & C

## Define BackBoneModule - A generic Module for all pretrained modules

In [7]:
class BackBoneModule(L.LightningModule):
  def __init__(self, backbone_module, num_classes: int):
    super().__init__()
    self.num_classes = num_classes
    self.criterion = nn.CrossEntropyLoss()
    self.backbone_module = backbone_module
    self._update_last_layer(num_classes) 
        
  def _update_last_layer(self, num_classes: int):
        if hasattr(self.backbone_module, 'classifier'):
            if isinstance(self.backbone_module.classifier, nn.Sequential):
                num_features = self.backbone_module.classifier[-1].in_features
                self.backbone_module.classifier[-1] = nn.Linear(num_features, num_classes)
            else:
                num_features = self.backbone_module.classifier.in_features
                self.backbone_module.classifier = nn.Linear(num_features, num_classes)
        elif hasattr(self.backbone_module, 'fc'):
            num_features = self.backbone_module.fc.in_features
            self.backbone_module.fc = nn.Linear(num_features, num_classes)

  def forward(self, x):
      return self.backbone_module(x)

  def training_step(self, batch, batch_idx):
      x, y = batch
      y_hat = self.forward(x)
      loss = self.criterion(y_hat, y)
      self.log("train_loss", loss)
      acc = (y_hat.argmax(dim=1) == y).float().mean()
      self.log('train_accuracy', acc, on_step=True, on_epoch=False, prog_bar=True)
      return loss

  def validation_step(self, batch, batch_idx):
      x, y = batch
      y_hat = self.forward(x)
      loss = self.criterion(y_hat, y)
      acc = (y_hat.argmax(dim=1) == y).float().mean()
      self.log_dict({"val_loss": loss, "val_accuracy": acc}, on_step=False, on_epoch=True)

  def test_step(self, batch, batch_idx):
      x, y = batch
      y_hat = self.forward(x)
      loss = self.criterion(y_hat, y)
      acc = (y_hat.argmax(dim=1) == y).float().mean()
      self.log_dict({"test_loss": loss, "test_accuracy": acc}, on_step=False, on_epoch=True)

  def configure_optimizers(self):
      optimizer = torch.optim.Adam(self.backbone_module.parameters(), lr=1e-4)
      return optimizer


## Load pretrained models - MobileNet, GoogleNet, ResNet50 & VGG19

In [8]:
pretrained_models = {
    'mobilenet': models.mobilenet_v2(pretrained=True),
    'googlenet': models.googlenet(pretrained=True),
    'resnet50': models.resnet50(pretrained=True),
    'vgg19': models.vgg19(pretrained=True)
}

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth
100%|██████████| 13.6M/13.6M [00:00<00:00, 73.0MB/s]
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and

## Create `train_model` function - Given a pretrained model, trains the model on the Birds Species Dataset and calculates the accuracy and loss over the test and validation sets

In [9]:
def train_model(model_name, pretrained_model, epochs=5):
    """
    Inputs:
        model_name - Name of the model you want to run. Is used to look up the class in "model_dict"
    """
    tensorboard = TensorBoardLogger("q3_logs", name=model_name)

    early_stop_callback = EarlyStopping(
            monitor='val_accuracy',    # Metric to monitor
            min_delta=0.00,       # Minimum change to qualify as an improvement
            patience=5,          # Number of epochs with no improvement after which training will be stopped
            verbose=True,
            mode='max'            # 'max' because we aim to maximize validation accuracy. Use 'min' for loss.
        )
    trainer = L.Trainer(
        accelerator="auto",
        devices=1,
        max_epochs=epochs,
        logger=tensorboard,
        callbacks=[ModelCheckpoint(dirpath="checkpoints/", save_top_k=1, monitor="val_loss"),
                  early_stop_callback]
    )

    L.seed_everything(42)  # To be reproducable
    model = BackBoneModule(backbone_module=pretrained_model, num_classes=525)
    trainer.fit(model, train_loader, val_loader)

    # Test best model on validation and test set
    val_result = trainer.test(model, dataloaders=val_loader, verbose=False)
    test_result = trainer.test(model, dataloaders=test_loader, verbose=False)

    return model, val_result[0], test_result[0]

## Iterate over the pretrained models - Run `train_model` on each of them

In [10]:
post_trained_models = {}
for model_name, pretrained_model in pretrained_models.items():
    print(f"Training {model_name}")
    model, val_result, test_result = train_model(model_name, pretrained_model, epochs=5)
    post_trained_models[model_name] = model
    print(f"Results for {model_name}: \n")
    print(f'\tValidation Results:\n\tAccuracy: {val_result["test_accuracy"]}, Loss: {val_result["test_loss"]}')
    print(f'\tTest Results:\n\tAccuracy: {test_result["test_accuracy"]}, Loss: {test_result["test_loss"]}')

Training mobilenet


INFO: GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO: Seed set to 42
2024-02-07 13:43:10.693390: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-07 13:43:10.693485: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-07 13:43:10.853759: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
INFO: 
  | Name            | Type             | Params
-----------------------------------------------------
0 | crite

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

INFO: Metric val_accuracy improved by 0.010 >= min_delta = 0.0. New best score: 0.934
INFO: `Trainer.fit` stopped: `max_epochs=5` reached.
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Testing: |          | 0/? [00:00<?, ?it/s]

INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Testing: |          | 0/? [00:00<?, ?it/s]

Results for vgg19: 

	Validation Results:
	Accuracy: 0.9344761967658997, Loss: 0.2460256963968277
	Test Results:
	Accuracy: 0.9577142596244812, Loss: 0.13696612417697906


# Section D - Use one of the trained models as a feature extractor

## Use MobileNet's last layer as a feature extractor for a Logistic Regression model

In [14]:
model = post_trained_models['mobilenet']
model.backbone_module.classifier = model.backbone_module.classifier[:-1]
model = model.to(device)

In [15]:
def extract_features(loader, model):
    model.eval()
    features = []
    labels = []
    with torch.no_grad():
        for x, y in tqdm(loader):
            x = x.to(device)
            y = y.to(device)
            feature = model(x)
            features.append(feature)
            labels.append(y)
    return torch.cat(features).cpu().numpy(), torch.cat(labels).cpu().numpy()

In [16]:
train_features, train_labels = extract_features(train_loader, model)
test_features, test_labels = extract_features(test_loader, model)
val_features, val_labels = extract_features(val_loader, model)

100%|██████████| 42/42 [00:04<00:00,  9.77it/s]


## Train the LogisticRegression model

In [17]:
clf = LogisticRegression(max_iter=1000)
clf.fit(train_features, train_labels)

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=1000)

## Predict the labels of the train, validation and test sets and print the accuracy scores

In [18]:
train_preds = clf.predict(train_features)
test_preds = clf.predict(test_features)
val_preds = clf.predict(val_features)

In [19]:
print(f"Train accuracy: {accuracy_score(train_labels, train_preds)}")
print(f"Test accuracy: {accuracy_score(test_labels, test_preds)}")
print(f"Validation accuracy: {accuracy_score(val_labels, val_preds)}")

Train accuracy: 1.0
Test accuracy: 0.9908571428571429
Validation accuracy: 0.9706666666666667
